In [1]:
import urllib.request 
from urllib.request import urlopen
import requests 
import re
import sqlite3

#saving the website to be scraped
myurl = 'https://money.cnn.com/data/hotstocks/'

#create an empty list to gather information later
stocks=[]


try: 
    hotstocks_handle=requests.get(myurl)          #get the url source code
    #Open the website source code in text format
    hotstocks_text=hotstocks_handle.text     

    these_regex="<a.+>(.+?)</a>"            #regex funtion to zone in on the information needed
    pattern=re.compile(these_regex)         #gather all tickers into a single list
        
        
    #slicing the source code for the section that we need
    part_text=hotstocks_text[hotstocks_text.find('Most Actives'):hotstocks_text.find('Gainers')] 

    #executing the regex funtion to find all tickers from the sliced section
    tickers=re.findall(pattern, part_text)
        
except:
    print('Could not load information at this time. Check back again later!')
    sys.exit(1)  #quit program if failed


with open('stocks.txt', 'w') as myfile:       #creating and opening the file in write mode to edit new information 
    for ticker in tickers:                    #iterate through each ticker collected
        stock_url='https://finance.yahoo.com/quote/'+ticker+'?p='+ticker+'&.tsrc=fin-srch-v1'   #custom url for each ticker
        stock_handle=requests.get(stock_url)
        stock_text=stock_handle.text
    
        close_value=re.findall('PREV_CLOSE-value.+?><.+?>(.+?)</span>', stock_text)           #re to find the exact price of stock
        for num in close_value:
            c_value=num                                                              #turning list into string
        
        open_value=re.findall('OPEN-value.+?><.+?>(.+?)</span>', stock_text)
        for num in open_value:
            o_value=num
        
        volume=re.findall('TD_VOLUME-value.+?><.+?>(.+?)</span>', stock_text)
        for num in volume:
            v_value=num.replace(',','')                                        #removing commas from the volume number
        
        pos=[ticker, o_value, c_value, v_value]                                #saving ticker name and all information in a list
        
        for elem in pos:
            myfile.write(elem+',')                                           #writing the list in the open text file with commas seperating each charachter
        
        stocks.append(pos)                                                  #adding each iteration to the main list

conn = sqlite3.connect('StocksDatabase.sqlite')                         #creating and opening database file
cur = conn.cursor()           

cur.execute('DROP TABLE IF EXISTS StocksTable')                      #deleting if the table already exists to put in fresh values
cur.execute('CREATE TABLE StocksTable (Ticker TEXT, OpenPrice REAL, PrevClose REAL, Volume INTEGER)')   #creating a table of 4 columns with all information needed

for stock in stocks:
    insertcmd= 'INSERT INTO StocksTable (Ticker, OpenPrice, PrevClose, Volume) VALUES (?,?,?,?)'       #adding each stock information as rows to respective columns
    cur.execute(insertcmd , (stock [0], stock [1], stock[2], stock[3]))

conn.commit()   #saving the database file
cur.close()     #closing the database file

print("Success! Program has been executed!")


Success! Program has been executed!
